In [ ]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import re

class ChemEngScraper:
    def __init__(self, base_url):
        """초기화 및 기본 설정"""
        self.base_url = base_url
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_0_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.277 Whale/2.9.118.38 Safari/537.36'
        }
        self.hrefs = []
        self.all_content_texts = []

    def get_lnb_links(self):
        """웹 페이지에서 lnb 요소 내부의 depth 링크를 추출"""
        driver = webdriver.Chrome()
        driver.get(self.base_url)
        
        # lnb 요소 내부의 class가 "depth"인 모든 a 태그에서 href 속성 추출
        lnb_element = driver.find_element("xpath", '//*[@id="lnb"]')
        depth_elements = lnb_element.find_elements("xpath", './/*[@class="depth"]//a')
        self.hrefs = [element.get_attribute("href") for element in depth_elements if element.get_attribute("href")]
        
        driver.quit()
    
    def fetch_content_from_url(self, url):
        """URL에서 id='cont' 요소의 텍스트 콘텐츠를 가져옵니다"""
        response = requests.get(url, headers=self.headers)
        if response.status_code != 200:
            print(f"Failed to retrieve {url}, status code: {response.status_code}")
            return None

        # BeautifulSoup을 사용해 HTML 파싱
        soup = BeautifulSoup(response.content, 'html.parser')
        content_div = soup.find(id="cont")
        
        if content_div:
            # 내부에서 class="hide"인 요소 제거
            for hidden in content_div.select(".hide"):
                hidden.extract()

            # 'inner_contents_title mt' 또는 'inner_contents_title mb' 요소의 앞뒤 \n 개수 맞추기
            self.adjust_special_elements(content_div)
            
            # 텍스트 내용에서 불필요한 줄바꿈과 공백 제거
            content_text = self.clean_text(content_div.text)
            return content_text
        return None

    def adjust_special_elements(self, content_div):
        """특정 클래스의 요소의 앞뒤 \n 개수를 1로 맞춤"""
        special_elements = content_div.find_all(class_=re.compile(r'inner_contents_title (mt|mb)'))
        for special in special_elements:
            if special.text:
                text = special.text.strip()
                # 앞뒤에 \n이 정확히 1개씩 되도록 조정
                if not text.startswith("\n"):
                    text = "\n" + text
                if not text.endswith("\n"):
                    text = text + "\n"
                special.string = text

    def clean_text(self, text):
        """텍스트에서 불필요한 줄바꿈과 공백을 정리"""
        # \n이 3개 이상 연속되면 2개로 줄이기
        text = re.sub(r'\n{3,}', '\n\n\n', text)
        # 캐리지 리턴 제거 및 여러 공백을 하나의 공백으로 줄이기
        return text.replace('\r', ' ').replace('  ', ' ').strip()

    def scrape_all_content(self):
        """모든 URL을 순회하며 콘텐츠를 수집"""
        self.get_lnb_links()
        for href in self.hrefs:
            content_text = self.fetch_content_from_url(href)
            if content_text:
                self.all_content_texts.append(content_text)
    
    def get_combined_content(self):
        """모든 콘텐츠를 하나의 텍스트로 결합"""
        return "\n\n".join(self.all_content_texts)

    def display_combined_content(self):
        """결합된 텍스트 콘텐츠를 출력"""
        combined_content = self.get_combined_content()
        # print("Combined Content from All URLs:")
        print(combined_content)
        print("\n" + "="*50 + "\n")  # 구분선



In [ ]:
# ChemEngScraper 클래스를 임포트했다고 가정하고 사용

# 기본 URL 설정
base_url = "https://chemeng.khu.ac.kr/chemeng/user/main/view.do"

# ChemEngScraper 객체 생성 및 데이터 스크래핑 실행
scraper = ChemEngScraper(base_url)

# 모든 콘텐츠를 스크래핑
scraper.scrape_all_content()

# 결합된 콘텐츠 출력
scraper.display_combined_content()